In [66]:
!pip install --force-reinstall -v "tensorflow==2.15.1"

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
  Obtaining dependency information for tensorflow==2.15.1 from https://files.pythonhosted.org/packages/c1/2d/636471492d93b6c1bf5375b81be7105a313a8c91a07c37e43625b00ca5c3/tensorflow-2.15.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependen

In [67]:
import tensorflow as tf

In [80]:
!python -V

Python 3.11.0rc1


In [68]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print("Найдено GPU:", gpu)
else:
    print("GPU не обнаружены")

Найдено GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Найдено GPU: PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
Найдено GPU: PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')
Найдено GPU: PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')
Найдено GPU: PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU')


In [69]:
from PIL import Image, ImageDraw, ImageFont
import os

In [70]:
data_dir = 'text'
wordlist_file = os.path.join(data_dir, 'sah.wordlist')
training_text_file = os.path.join(data_dir, 'sah.training_text')
fonts_list_file = os.path.join(data_dir, 'okfonts_test.txt')

fonts_dir = os.path.join(data_dir, 'fonts')

output_dir = 'test_images'

In [71]:
with open(wordlist_file, 'r', encoding='utf-8') as f:
    words = f.readlines()

with open(training_text_file, 'r', encoding='utf-8') as f:
    training_text = f.readlines()

with open(fonts_list_file, 'r', encoding='utf-8') as f:
    styles = [line.strip() for line in f.readlines()]

In [72]:
def load_fonts_from_styles(styles_list, base_fonts_dir):
    fonts = []
    for style in styles_list:
        style_dir = os.path.join(base_fonts_dir, style)
        if os.path.exists(style_dir):
            for file in os.listdir(style_dir):
                if file.endswith(('.ttf', '.otf')):
                    fonts.append(os.path.join(style_dir, file))
        else:
            print(f"Директория для стиля '{style}' не найдена: {style_dir}")
    return fonts

In [73]:
fonts = load_fonts_from_styles(styles, fonts_dir)

In [74]:
def generate_image(text, font_path, image_size=(500, 100), font_size=30):
    image = Image.new('RGB', image_size, color=(255, 255, 255))
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        print(f"Не удалось загрузить шрифт: {font_path}")
        return None
        
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    text_x = (image_size[0] - text_width) // 2
    text_y = (image_size[1] - text_height) // 2

    draw.text((text_x, text_y), text, font=font, fill=(0, 0, 0))
    
    return image

In [75]:
for i, text in enumerate(training_text):
    if i < len(fonts):
        font = fonts[i]
        image = generate_image(text.strip(), font)
        if image:
            output_path = os.path.join(output_dir, f'image_{i}_{os.path.basename(font)}.png')
            image.save(output_path)
            print(f'Изображение сохранено: {output_path}')
    else:
        print("Недостаточно шрифтов для всех текстов.")
        break

Изображение сохранено: test_images/image_0_AlegreyaSans-ThinItalic.ttf.png
Изображение сохранено: test_images/image_1_AlegreyaSans-ExtraBold.ttf.png
Изображение сохранено: test_images/image_2_AlegreyaSans-MediumItalic.ttf.png
Изображение сохранено: test_images/image_3_AlegreyaSans-Regular.ttf.png
Изображение сохранено: test_images/image_4_AlegreyaSans-Light.ttf.png
Изображение сохранено: test_images/image_5_AlegreyaSans-BlackItalic.ttf.png
Изображение сохранено: test_images/image_6_AlegreyaSans-LightItalic.ttf.png
Изображение сохранено: test_images/image_7_AlegreyaSans-Black.ttf.png
Изображение сохранено: test_images/image_8_AlegreyaSans-BoldItalic.ttf.png
Изображение сохранено: test_images/image_9_AlegreyaSans-Medium.ttf.png
Изображение сохранено: test_images/image_10_AlegreyaSans-Bold.ttf.png
Изображение сохранено: test_images/image_11_AlegreyaSans-Thin.ttf.png
Изображение сохранено: test_images/image_12_AlegreyaSans-Italic.ttf.png
Изображение сохранено: test_images/image_13_Alegreya

In [76]:
import keras_ocr
import matplotlib.pyplot as plt

In [77]:
pipeline = keras_ocr.pipeline.Pipeline()

Looking for /root/.keras-ocr/craft_mlt_25k.h5


ValueError: Unrecognized keyword arguments passed to Dense: {'weights': [array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
   

In [78]:
import matplotlib.pyplot as plt

import keras_ocr

# keras-ocr will automatically download pretrained
# weights for the detector and recognizer.
pipeline = keras_ocr.pipeline.Pipeline()

# Get a set of three example images
images = [
    keras_ocr.tools.read(url) for url in [
        'https://upload.wikimedia.org/wikipedia/commons/b/bd/Army_Reserves_Recruitment_Banner_MOD_45156284.jpg',
        'https://upload.wikimedia.org/wikipedia/commons/e/e8/FseeG2QeLXo.jpg',
        'https://upload.wikimedia.org/wikipedia/commons/b/b4/EUBanana-500x112.jpg'
    ]
]

# Each list of predictions in prediction_groups is a list of
# (word, box) tuples.
prediction_groups = pipeline.recognize(images)

# Plot the predictions
fig, axs = plt.subplots(nrows=len(images), figsize=(20, 20))
for ax, image, predictions in zip(axs, images, prediction_groups):
    keras_ocr.tools.drawAnnotations(image=image, predictions=predictions, ax=ax)

Looking for /root/.keras-ocr/craft_mlt_25k.h5


ValueError: Unrecognized keyword arguments passed to Dense: {'weights': [array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
   